In [ ]:
pip install rarfile

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import os
import rarfile
import cv2
from google.colab import drive
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from time import sleep
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2

from tensorflow.keras.layers import Conv1D,Reshape, MaxPooling1D, Flatten, Dense, Conv2D,LSTM, MaxPooling2D,Dropout
from tensorflow.keras import backend as K

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from PIL import Image
from tqdm.notebook import tqdm
import io


# mounting drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/Assignments/emotion_recognition_clip/dataset_zips'




Mounted at /content/drive


In [ ]:
def mean_absolute_deviation(y_true, y_pred):
    """
    Compute the Mean Absolute Deviation (MAD) between true and predicted values.

    Parameters:
    y_true (tensor): True labels.
    y_pred (tensor): Predicted values.

    Returns:
    tensor: The MAD metric.
    """
    return 1 - K.mean(K.abs(y_pred - y_true), axis=-1)

In [ ]:
# required input files to run the notebook
# train_image_names.npy, test_image_names.npy, train_image_embeds.rar

In [ ]:
# extracting image index for train and test datasets
file_names = np.load(os.path.join(data_path,'train_image_names.npy'))
file_names_test = np.load(os.path.join(data_path,'test_image_names.npy'))
file_names_test.shape

(20000,)

In [ ]:
# extracting image embeddings from CLIP ViT-L/14 model(.pt pytorch files)
rarfile_path = os.path.join(data_path, 'train_image_embeds.rar')
train_images_rar = rarfile.RarFile(rarfile_path)
train_feat_files = train_images_rar.infolist()
train_feat_files = [x.filename for x in train_feat_files]

# extracting image embeddings from CLIP ViT-L/14 model(.pt pytorch files test)
rarfile_path_test = os.path.join(data_path, 'FEATURES_TEST')
list_files_test = os.listdir(rarfile_path_test)


In [ ]:
print(" Reading image embed files Train")
all_files = []
for file_ in tqdm(range(len(train_feat_files)-1)):
    file_name = 'train_image_embeds/feat_image_batch_{}.pt'.format(file_)
    if file_name.endswith('.pt'):
      image_feat = train_images_rar.read(file_name)
      image_feat =  torch.load(io.BytesIO(image_feat))
      all_files.append(image_feat)
    else:
      print (file_.filename)

print(" Reading image embed files Test")
all_files_test = []
for file_ in tqdm(range(len(list_files_test))):
    file_name = 'feat_image_batch_{}.pt'.format(file_)
    if file_name.endswith('.pt'):
      image_feat =  torch.load(os.path.join(rarfile_path_test, file_name))

      all_files_test.append(image_feat)



#concatinating all image features
all_torch = torch.cat(all_files)
all_torch_test = torch.cat(all_files_test)

 Reading image embed files Train


  0%|          | 0/362 [00:00<?, ?it/s]

 Reading image embed files Test


  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
#extraciting every 10th frame name for video index and removing "frame_" for train
all_torch_index = file_names[:all_torch.shape[0]:10]
index_names = [x.split('_frame')[0].replace('_','') for x in all_torch_index]

#extraciting every 10th frame name for video index and removing "frame_" for train
all_torch_index_test = file_names_test[:all_torch_test.shape[0]:10]
index_names_test = [x.split('_frame')[0].replace('_','') for x in all_torch_index_test]


##flattening data and reshaping it to required format for LSTM (no of inputs, timeframes, feature size)
flatten_torch = all_torch.flatten()
reshape_torch = flatten_torch.reshape((int(all_torch.shape[0]/10), 10,768))

flatten_torch_test = all_torch_test.flatten()
reshape_torch_test = flatten_torch_test.reshape((int(all_torch_test.shape[0]/10), 10,768))


In [ ]:
# reading ground truth annotations for train and test and changing it to pandas format for easier processing
annotations_train = pd.read_pickle(os.path.join(data_path, 'annotation_training.pkl'))
annotations_test = pd.read_pickle(os.path.join(data_path, 'annotation_test.pkl'))
classify_labels = list(annotations_test.keys()) # capturing output classes
ground_truth = pd.DataFrame(annotations_test).reset_index()
ground_truth['image_name'] = ground_truth['index'].map(lambda x: '.'.join(x.split('.')[:-1]))
ground_truth['image_name'] = ground_truth['image_name'].map(lambda x: x.replace('_',''))
ground_truth = ground_truth.drop(['index'], axis = 1)
ground_truth = ground_truth.set_index(['image_name'])
ground_truth = ground_truth.loc[index_names_test].drop(["interview"],axis = 1)


ground_truth_train = pd.DataFrame(annotations_train).reset_index()
ground_truth_train['image_name'] = ground_truth_train['index'].map(lambda x: '.'.join(x.split('.')[:-1]))
ground_truth_train['image_name'] = ground_truth_train['image_name'].map(lambda x: x.replace('_',''))
ground_truth_train = ground_truth_train.drop(['index'], axis = 1)
ground_truth_train = ground_truth_train.set_index(['image_name'])
ground_truth_train = ground_truth_train.loc[index_names].drop(["interview"],axis =1)


In [ ]:
# converting inputs to numpy arrrays for pytorch model input
y_train = ground_truth_train.to_numpy()
X_train = reshape_torch.numpy()
y_test = ground_truth.to_numpy()
X_test = reshape_torch_test.numpy()

In [ ]:
# X_train = (X_train - X_train.mean())/X_train.std()
# X_test = (X_test - X_train.mean())/X_train.std()

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((3620, 10, 768), (3620, 5), (2000, 10, 768), (2000, 5))

In [ ]:
def metrics_calc(pred, act):
    print ('*'*100)
    print('Evaluating mean accuracy on test data \n')
    acc = []
    # calculating mean accuracy for each trait
    for column in pred:
        # print (column)
        # acc_class = 1 - ((abs(pred[column] - act[column])).mean()/abs(act[column] - act[column].mean()).sum())
        acc_class = 1 - ((abs(pred[column] - act[column])).mean())
        print ('accuracy for {} is {}'.format(column, acc_class))
        acc.append(acc_class)
    print ("mean accuracy for all classes {}".format(np.mean(acc)))

def estimate_accuracy(predictions_model, actuals= ground_truth, column_names = list(annotations_test.keys())[:4] +[list(annotations_test.keys())[-1]], audio_test_names = index_names_test):
    test_names = [x for x in audio_test_names]
    predictions = pd.DataFrame(predictions_model, columns =column_names, index = test_names)
    predictions = predictions.reset_index()
    predictions['image_name'] = predictions['index'].map(lambda x: x) # removing frame number to get only the image name
    predictions = predictions.drop(['index'], axis = 1)
    predictions = predictions.groupby(['image_name']).mean() # average score at image level (grouping all frames result into single result)
    metrics_calc(predictions.sort_index(), actuals.sort_index())

# Feed Forward Neural Network

In [ ]:
def build_model(input_shape, num_classes):
    """
    base model, flattening data and passing it to dense layer
    """
    model = Sequential()
    model.add(Flatten())

    # Dense layers
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='sigmoid'))  # Sigmoid for multi-label classification

    return model

num_classes = y_train.shape[1]  # Number of labels

model = build_model((10, 768), num_classes)
model.compile(
    optimizer=Adam(),
    loss='mse',  # Binary cross-entropy for multi-label classification
    metrics=[mean_absolute_deviation]
)
# Train the model

history = model.fit(
    X_train, y_train,
    epochs=10,  # Number of epochs
    batch_size=128,  # Batch size
    validation_split=0.1  # Split data for validation
)

print("printing predicion accuracy for trained model")
y_predict =model.predict(X_test)

estimate_accuracy(y_predict)

Epoch 1/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 0.1007 - mean_absolute_deviation: 0.7358 - val_loss: 0.0528 - val_mean_absolute_deviation: 0.8297
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0351 - mean_absolute_deviation: 0.8628 - val_loss: 0.0156 - val_mean_absolute_deviation: 0.9004
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0127 - mean_absolute_deviation: 0.9104 - val_loss: 0.0148 - val_mean_absolute_deviation: 0.9033
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0106 - mean_absolute_deviation: 0.9180 - val_loss: 0.0141 - val_mean_absolute_deviation: 0.9053
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0101 - mean_absolute_deviation: 0.9199 - val_loss: 0.0139 - val_mean_absolute_deviation: 0.9062
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0092 - mean_absolute_deviation: 0.9233 - val_loss: 0.0138 - val_mean_absolute_deviation: 0.9068
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss:

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 7680)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         983,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,951,441 (11.26 MB)

 Trainable params: 983,813 (3.75 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,967,628 (7.51 MB)

# LSTM Model

In [ ]:
import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define the model
model = Sequential()

# LSTM Layer
model.add(LSTM(128, input_shape=(10, 768), return_sequences=False))  # You can adjust units

# Dropout for regularization
model.add(Dropout(0.3))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Output layer with 5 outputs for class probabilities
model.add(Dense(5, activation='sigmoid'))  # Sigmoid activation for multi-label classification

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Model Summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 128)                 │         459,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 5)                   │             325 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 467,845 (1.78 MB)

 Trainable params: 467,845 (1.78 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])


Epoch 1/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - loss: 0.0186 - mae: 0.1078 - val_loss: 0.0141 - val_mae: 0.0949
Epoch 2/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 8s 79ms/step - loss: 0.0116 - mae: 0.0861 - val_loss: 0.0130 - val_mae: 0.0911
Epoch 3/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - loss: 0.0099 - mae: 0.0795 - val_loss: 0.0128 - val_mae: 0.0901
Epoch 4/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - loss: 0.0093 - mae: 0.0773 - val_loss: 0.0129 - val_mae: 0.0910
Epoch 5/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - loss: 0.0079 - mae: 0.0709 - val_loss: 0.0130 - val_mae: 0.0914
Epoch 6/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 0.0078 - mae: 0.0705 - val_loss: 0.0133 - val_mae: 0.0917
Epoch 7/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - loss: 0.0073 - mae: 0.0674 - val_loss: 0.0131 - val_mae: 0.0913
Epoch 8/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - loss: 0.0068 - mae: 0.0656 - val_loss: 0.0130 - val_mae: 0.0907


In [ ]:

y_predict =model.predict(X_test)

estimate_accuracy(y_predict)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
****************************************************************************************************
Evaluating mean accuracy on test data 

accuracy for extraversion is 0.9115205880604057
accuracy for neuroticism is 0.9072131643816829
accuracy for agreeableness is 0.9105659773077119
accuracy for conscientiousness is 0.919642971279624
accuracy for openness is 0.9097282190634145
mean accuracy for all classes 0.9117341840185678


# LSTM Model - Experiment 2

In [ ]:
from tensorflow.keras.layers import BatchNormalization
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)


In [ ]:
model = Sequential()

# First LSTM Layer with Batch Normalization and Dropout
model.add(LSTM(128, input_shape=(10, 768), return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.4))

# Second LSTM Layer with Dropout
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.3))

# Fully connected layer with L2 regularization
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))

# Output layer with 5 outputs for class probabilities
model.add(Dense(5, activation='sigmoid'))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='mse', metrics=['mae'])

# Train with validation split and learning rate scheduler
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
                               lr_scheduler])


# Evaluate the model

y_predict =model.predict(X_test)

estimate_accuracy(y_predict)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 13s 63ms/step - loss: 0.0911 - mae: 0.1364 - val_loss: 0.0789 - val_mae: 0.1151 - learning_rate: 1.0000e-04
Epoch 2/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 8s 84ms/step - loss: 0.0776 - mae: 0.1139 - val_loss: 0.0705 - val_mae: 0.1044 - learning_rate: 1.0000e-04
Epoch 3/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 10s 106ms/step - loss: 0.0694 - mae: 0.1038 - val_loss: 0.0642 - val_mae: 0.0988 - learning_rate: 1.0000e-04
Epoch 4/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 9s 93ms/step - loss: 0.0626 - mae: 0.0975 - val_loss: 0.0592 - val_mae: 0.0967 - learning_rate: 1.0000e-04
Epoch 5/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 7s 60ms/step - loss: 0.0573 - mae: 0.0935 - val_loss: 0.0546 - val_mae: 0.0953 - learning_rate: 1.0000e-04
Epoch 6/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 65ms/step - loss: 0.0520 - mae: 0.0897 - val_loss: 0.0506 - val_mae: 0.0943 - learning_rate: 1.0000e-04
Epoch 7/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 73ms/step - loss: 0.0478 - mae: 0.0878 - val_loss: 0.0470 - val_mae: 0.0939 -

# CNN Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout, BatchNormalization

# Define the model
model = Sequential()

# Convolutional Layer 1
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(10, 768)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Convolutional Layer 2
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Flatten the output from the conv layers
model.add(Flatten())

# Fully connected layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

# Output layer with 5 outputs for class probabilities
model.add(Dense(5, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Model summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 8, 64)               │         147,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 8, 64)               │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 8, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 6, 128)              │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 6, 128)              │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 6, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 768)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 64)                  │          49,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 5)                   │             325 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 222,533 (869.27 KB)

 Trainable params: 222,149 (867.77 KB)

 Non-trainable params: 384 (1.50 KB)

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])


# Evaluate the model

y_predict =model.predict(X_test)

estimate_accuracy(y_predict)

Epoch 1/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.0826 - mae: 0.2365 - val_loss: 0.0291 - val_mae: 0.1366
Epoch 2/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.0470 - mae: 0.1736 - val_loss: 0.0197 - val_mae: 0.1113
Epoch 3/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0288 - mae: 0.1349 - val_loss: 0.0157 - val_mae: 0.0998
Epoch 4/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0203 - mae: 0.1136 - val_loss: 0.0147 - val_mae: 0.0968
Epoch 5/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0165 - mae: 0.1025 - val_loss: 0.0146 - val_mae: 0.0965
Epoch 6/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0152 - mae: 0.0987 - val_loss: 0.0143 - val_mae: 0.0954
Epoch 7/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0135 - mae: 0.0928 - val_loss: 0.0144 - val_mae: 0.0958
Epoch 8/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0130 - mae: 0.0910 - val_loss: 0.0143 - val_mae: 0.0956
Epoch 9/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.012

# CNN-LSTM Hybrid Model

In [ ]:
model = Sequential()

# 1D Convolutional Layer to capture local patterns
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(10, 768)))
model.add(Dropout(0.3))

# LSTM Layer to capture temporal dependencies
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.3))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Output layer
model.add(Dense(5, activation='sigmoid'))

# Compile
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])


# Evaluate the model

y_predict =model.predict(X_test)

estimate_accuracy(y_predict)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - loss: 0.0184 - mae: 0.1085 - val_loss: 0.0137 - val_mae: 0.0931
Epoch 2/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 0.0126 - mae: 0.0899 - val_loss: 0.0137 - val_mae: 0.0927
Epoch 3/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 0.0109 - mae: 0.0825 - val_loss: 0.0137 - val_mae: 0.0925
Epoch 4/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - loss: 0.0103 - mae: 0.0808 - val_loss: 0.0131 - val_mae: 0.0906
Epoch 5/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - loss: 0.0091 - mae: 0.0761 - val_loss: 0.0138 - val_mae: 0.0929
Epoch 6/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0087 - mae: 0.0742 - val_loss: 0.0134 - val_mae: 0.0921
Epoch 7/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.0079 - mae: 0.0709 - val_loss: 0.0146 - val_mae: 0.0956
Epoch 8/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0079 - mae: 0.0707 - val_loss: 0.0142 - val_mae: 0.0941
Epoch 9/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.007

# Bidirectional LSTM Model

In [ ]:
from tensorflow.keras.layers import Bidirectional

model = Sequential()

# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(128, return_sequences=False), input_shape=(10, 768)))
model.add(Dropout(0.3))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Output layer with 5 outputs for class probabilities
model.add(Dense(5, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])


history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])


y_predict =model.predict(X_test)

estimate_accuracy(y_predict)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 13s 102ms/step - loss: 0.0196 - mae: 0.1098 - val_loss: 0.0151 - val_mae: 0.0984
Epoch 2/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 6s 71ms/step - loss: 0.0114 - mae: 0.0855 - val_loss: 0.0133 - val_mae: 0.0927
Epoch 3/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 12s 89ms/step - loss: 0.0098 - mae: 0.0791 - val_loss: 0.0134 - val_mae: 0.0927
Epoch 4/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 8s 85ms/step - loss: 0.0084 - mae: 0.0726 - val_loss: 0.0135 - val_mae: 0.0927
Epoch 5/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 8s 84ms/step - loss: 0.0081 - mae: 0.0715 - val_loss: 0.0146 - val_mae: 0.0963
Epoch 6/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 98ms/step - loss: 0.0075 - mae: 0.0685 - val_loss: 0.0143 - val_mae: 0.0955
Epoch 7/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 7s 72ms/step - loss: 0.0070 - mae: 0.0663 - val_loss: 0.0135 - val_mae: 0.0928
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step
****************************************************************************************************
Evaluating mean accuracy on test dat

# Bidirectional LSTM - Experiment 2

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define the model
model = Sequential()

# Bidirectional LSTM Layer 1
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(10, 768)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Bidirectional LSTM Layer 2
model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Fully connected layer
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Output layer with 5 outputs for class probabilities
model.add(Dense(5, activation='sigmoid'))

# Compile the model with a more sophisticated optimizer
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='mse', metrics=['mae'])

# Model summary
model.summary()

# Callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the model
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stopping, reduce_lr])

y_predict =model.predict(X_test)

estimate_accuracy(y_predict)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_1 (Bidirectional)      │ (None, 10, 256)             │         918,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 10, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 10, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 128)                 │         164,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,102,085 (4.20 MB)

 Trainable params: 1,101,061 (4.20 MB)

 Non-trainable params: 1,024 (4.00 KB)

Epoch 1/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 22s 149ms/step - loss: 0.0964 - mae: 0.2601 - val_loss: 0.0356 - val_mae: 0.1514 - learning_rate: 1.0000e-04
Epoch 2/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 17s 111ms/step - loss: 0.0800 - mae: 0.2349 - val_loss: 0.0388 - val_mae: 0.1569 - learning_rate: 1.0000e-04
Epoch 3/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 22s 132ms/step - loss: 0.0733 - mae: 0.2256 - val_loss: 0.0359 - val_mae: 0.1520 - learning_rate: 1.0000e-04
Epoch 4/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 21s 135ms/step - loss: 0.0660 - mae: 0.2131 - val_loss: 0.0361 - val_mae: 0.1520 - learning_rate: 1.0000e-04
Epoch 5/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 13s 141ms/step - loss: 0.0634 - mae: 0.2081 - val_loss: 0.0372 - val_mae: 0.1536 - learning_rate: 1.0000e-04
Epoch 6/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 21s 142ms/step - loss: 0.0594 - mae: 0.2013 - val_loss: 0.0359 - val_mae: 0.1511 - learning_rate: 1.0000e-04
Epoch 7/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 21s 153ms/step - loss: 0.0551 - mae: 0.1929 - val_loss: 0.0343 - val_mae:

# GRU Model

In [ ]:
from tensorflow.keras.layers import GRU

model = Sequential()

# GRU Layer
model.add(GRU(128, input_shape=(10, 768), return_sequences=False))
model.add(Dropout(0.3))

# Dense Layer
model.add(Dense(64, activation='relu'))

# Output Layer
model.add(Dense(5, activation='sigmoid'))

# Compile
model.compile(optimizer='adam', loss='mse', metrics=['mae'])


# Model summary
model.summary()

# Callbacks for training
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the model
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stopping, reduce_lr])

y_predict =model.predict(X_test)

estimate_accuracy(y_predict)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 128)                 │         344,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 5)                   │             325 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 353,413 (1.35 MB)

 Trainable params: 353,413 (1.35 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 9s 58ms/step - loss: 0.0223 - mae: 0.1184 - val_loss: 0.0140 - val_mae: 0.0948 - learning_rate: 0.0010
Epoch 2/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - loss: 0.0127 - mae: 0.0903 - val_loss: 0.0132 - val_mae: 0.0916 - learning_rate: 0.0010
Epoch 3/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - loss: 0.0112 - mae: 0.0845 - val_loss: 0.0130 - val_mae: 0.0912 - learning_rate: 0.0010
Epoch 4/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 9s 51ms/step - loss: 0.0101 - mae: 0.0799 - val_loss: 0.0128 - val_mae: 0.0904 - learning_rate: 0.0010
Epoch 5/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 6s 60ms/step - loss: 0.0095 - mae: 0.0774 - val_loss: 0.0127 - val_mae: 0.0901 - learning_rate: 0.0010
Epoch 6/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - loss: 0.0085 - mae: 0.0730 - val_loss: 0.0130 - val_mae: 0.0910 - learning_rate: 0.0010
Epoch 7/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - loss: 0.0084 - mae: 0.0727 - val_loss: 0.0141 - val_mae: 0.0938 - learning_rate: 0.0010
Epoch